# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
map_data = pd.read_csv('output.csv')
map_data

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country
0,Iqaluit,63.7506,-68.5145,21.20,86,75,18.41,CA
1,Salalah,17.0151,54.0924,68.00,68,0,3.44,OM
2,Qaanaaq,77.4840,-69.3632,-1.25,79,100,7.56,GL
3,Guerrero Negro,27.9769,-114.0611,59.83,71,0,15.30,MX
4,Ushuaia,-54.8000,-68.3000,46.40,76,40,27.63,AR
...,...,...,...,...,...,...,...,...
577,Koumac,-20.5667,164.2833,82.89,65,22,6.78,NC
578,Nanyang,32.9947,112.5328,53.01,38,100,7.70,CN
579,Arnedo,42.2167,-2.1000,50.00,87,0,11.50,ES
580,Andrews,32.3187,-102.5457,50.00,29,1,5.75,US


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations = map_data[['Latitude', 'Longitude']].astype(float)
humidity_heatmap = map_data['Humidity'].astype(float)

In [4]:
gmaps.configure(api_key=g_key)

In [5]:
# figure_layout = {
#     'width': '400px',
#     'height': '300px',
#     'border': '1px solid black',
#     'padding': '1px',
#     'margin': '0 auto 0 auto'
# }

In [6]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_heatmap, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)


fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
temp_narrow = map_data.loc[map_data['Max Temp'] >= 70]
temp_narrow2 = temp_narrow.loc[temp_narrow['Max Temp'] < 80]
wind_narrow = temp_narrow2.loc[temp_narrow2['Cloudiness'] == 0]
hotel_temp = wind_narrow.loc[wind_narrow['Wind Speed'] < 10]
hotel_df = hotel_temp.dropna()
hotel_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country
43,Kharagpur,22.3333,87.3333,70.66,33,0,5.06,IN
184,Panabá,21.2833,-88.2667,70.43,89,0,9.51,MX
261,Palāsa,18.7667,84.4167,75.15,39,0,2.37,IN
292,Tura,25.5198,90.2201,71.96,37,0,2.24,IN
294,Lālmohan,22.3384,90.7419,70.84,50,0,3.09,BD
376,Prieska,-29.6641,22.7474,74.55,27,0,4.85,ZA
384,Córdoba,-31.4135,-64.1811,73.00,88,0,6.91,AR
447,Pisco,-13.7000,-76.2167,71.60,78,0,5.75,PE
453,Hong Kong,22.2855,114.1577,72.00,67,0,1.01,HK
467,Assaí,-23.3733,-50.8414,73.40,64,0,6.91,BR


In [8]:
# Add marker layer ontop of heat map


# Display figure


In [9]:
hotel_df = hotel_df.reindex(columns = hotel_df.columns.tolist() + ['Hotel Name'])


In [10]:
# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
#     base_url = "https://maps.googleapis.com/maps/api/place/textsearch/json?"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
#     print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

In [11]:
hotel_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Hotel Name
43,Kharagpur,22.3333,87.3333,70.66,33,0,5.06,IN,Hotel Wonder Inn (Kharagpur)
184,Panabá,21.2833,-88.2667,70.43,89,0,9.51,MX,Cenote & Cabañas La Xtabay
261,Palāsa,18.7667,84.4167,75.15,39,0,2.37,IN,SPR Grand Hotel
292,Tura,25.5198,90.2201,71.96,37,0,2.24,IN,"Dongrama Home Stay, Tura Meghalaya"
294,Lālmohan,22.3384,90.7419,70.84,50,0,3.09,BD,Hotel Grand Park Barisal
376,Prieska,-29.6641,22.7474,74.55,27,0,4.85,ZA,Excelsior River Cottage
384,Córdoba,-31.4135,-64.1811,73.00,88,0,6.91,AR,Constantino Hostel
447,Pisco,-13.7000,-76.2167,71.60,78,0,5.75,PE,"Hotel Paracas, a Luxury Collection Resort, Par..."
453,Hong Kong,22.2855,114.1577,72.00,67,0,1.01,HK,The OTTO Hotel
467,Assaí,-23.3733,-50.8414,73.40,64,0,6.91,BR,Aguativa Golf Resort


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
locations = hotel_df[["Latitude", "Longitude"]].astype(float)
humidity = hotel_df["Humidity"].astype(float)

In [14]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [15]:
hotels_list = hotel_df["Hotel Name"].tolist()

In [16]:
hotel_markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel: {hotel_info}" for hotel in hotels_list]
)


fig = gmaps.figure()
fig.add_layer(hotel_markers)

fig

Figure(layout=FigureLayout(height='420px'))

In [17]:
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(hotel_markers)

fig

Figure(layout=FigureLayout(height='420px'))